# 7. 데이터 준비하기: 다듬기, 변형, 병합

# 7.1 데이터합치기

## 7.1.1 데이터베이스 스타일로 DataFrame 합치기

In [167]:
import pandas as pd
from pandas import DataFrame

In [168]:
df1 = DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2= DataFrame({'key':['a','b','d'],'data2':range(3)})

In [169]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [170]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [171]:
pd.merge(df1,df2) #merge함수를 이용하여 df1과 df2를 merge 
#(키값을 중심으로 data1,2를 가지고 온다. 겹치는 key값만 가지고 옴)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [172]:
pd.merge(df1,df2,on='key') #merge와의 다른점은 merge는 겹치는 칼럼의 이름을 키로 사용하지만, 
#이 예에서처럼 명시적으로 하는것이 좋은예

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- join 연산과 비슷

---------
- 두 객체에 공통되는 칼럼 이름이 하나도 없으면 따로 지정해주면 된다.

In [173]:
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4 = DataFrame({'rkey':['a','b','d'],'data1':range(3)})

In [174]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,data1_x,lkey,data1_y,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


- 앞의 결과에서 보면 'c'와 'd'의 해당값이 빠진것을 볼 수 있다.
- merge함수의 경우엔 교집합인 결과를 반환
- how인자로 'left', 'right', 'outer를 넘겨서 각각 왼쪽, 오른쪽, 완전 외부 조인을 수행할 수 있다.

In [175]:
pd.merge(df1,df2,how='outer') #합집합 개념 (외부조인)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,3,c,NaN
7,NaN,d,2


- outer join은 left와 right를 합친 것 (full join)

In [176]:
pd.merge(df1,df2,how='left') #합집합 개념

,data1,key,data2
0,0,b,1
1,1,b,1
2,2,a,0
3,3,c,NaN
4,4,a,0
5,5,a,0
6,6,b,1


---------------------

In [181]:
#다대다 병합
df1 = DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})
df2 = DataFrame({'key':['a','b','a','b','d'],'data2':range(5)})

In [182]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [183]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [184]:
pd.merge(df1, df2, on='key', how='left')  #왼쪽우선 외부조인

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,2,a,0
5,2,a,2
6,3,c,NaN
7,4,a,0
8,4,a,2
9,5,b,1


In [185]:
pd.merge(df1, df2, on='key', how='outer')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [187]:
pd.merge(df1, df2, how='inner') 
#다 대 다 조인은 두 로우의 데카르트 곱을 반환한다.

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


---------------
- 여러 개의 키를 병합하려면 칼럼 이름이 들어간 리스트를 넘기면 된다.

In [188]:
left = DataFrame({'key1':['foo','foo','bar'],
                  'key2':['one','two','one'],
                  'lval':[1,2,3]})

In [189]:
right = DataFrame({'key1':['foo','foo','bar','bar'],
                   'key2':['one','one','one','two'],
                   'rval':[4,5,6,7]})

In [162]:
pd.merge(left, right, on=['key1','key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6
4,bar,two,NaN,7


- 칼럼과 칼럼을 조인할 때 전달한 DataFrame 객체의 색인은 무시된다.

--------------
- 머지 연산에서 고려해야 할 마지막 사항은 겹치는 칼럼 이름에 대한 처리이다.
- merge함수에 있는 suffixes 인자를 통해 겹치는 칼럼 이름 뒤에 붙일 문자열을 지정해 줄 수 있다.

In [163]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [164]:
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## 7.1.2 색인 머지하기

- 머지하려는 키가 DataFrame의 색인일 수 있다.
- 이때에는 left_index=True 혹은 right_index=True 옵션을 지정해 해당 색인을 머지 키로 사용할 수 있다.

In [19]:
left1 = DataFrame({'key':['a','b','a','a','b','c'], 'value':range(6)})

In [20]:
right1 = DataFrame({'group_val':[3.5,7]},index=['a','b'])

In [21]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [22]:
right1

,group_val
a,3.5
b,7.0


- left1의 key와 light1의 index를 merge해야한다. 그러기위해 left_on 사용

In [23]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [24]:
pd.merge(left1,right1,left_on='key',right_index=True,how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


-------------
- 계층 색인된 데이터는 약간 복잡하다.

In [190]:
import numpy as np
lefth=DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                 'key2':[2000,2001,2002,2001,2002],
                 'data':np.arange(5.)})

In [191]:
righth=DataFrame(np.arange(12).reshape((6,2)),
                 index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                        [2001,2000,2000,2000,2001,2002]],
                 columns=['event1','event2'])

In [27]:
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [28]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [29]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


In [30]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True,how='outer')

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1
4,4,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2,3


- 양쪽에 공통으로 있는 여러 개의 색인을 머지하는 것은 일도 아니다.

In [192]:
left2 = DataFrame([[1.,2.],[3.,4],[5.,6.]],
                  index=['a','c','e'],
                  columns=['Ohio','Nevada'])

In [193]:
right2 = DataFrame([[7.,8.],[9.,10],[11.,12.],[13.,14.]],
                   index=['b','c','d','e'],
                   columns=['Missouri','Alabama'])

In [194]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [195]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [35]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


- join 메서드를 사용하면 편리하다
- join 메서드는 칼럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다.

In [196]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


In [197]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


- join과 다르게 index기준으로 병합해준다.

- 색인 대 색인으로 두 DataFrame을 합치려면 그냥 간단한 머지하려는 DataFrame의 리스트를 join 메서드에 넘기면 된다.

In [200]:
another = DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                  index=['a','c','e','f'],
                  columns=['New York','Oregon'])

In [202]:
another

,New York,Oregon
a,7,8
c,9,10
e,11,12
f,16,17


In [208]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [209]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [203]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
c,3,4,9,10,9,10
e,5,6,13,14,11,12


In [204]:
right2.join([left2, another])

,Missouri,Alabama,Ohio,Nevada,New York,Oregon
b,7,8,NaN,NaN,NaN,NaN
c,9,10,3,4,9,10
d,11,12,NaN,NaN,NaN,NaN
e,13,14,5,6,11,12


In [205]:
left2.join([right2,another],how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,NaN,NaN,7,8
b,NaN,NaN,7,8,NaN,NaN
c,3,4,9,10,9,10
d,NaN,NaN,11,12,NaN,NaN
e,5,6,13,14,11,12
f,NaN,NaN,NaN,NaN,16,17


In [207]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


## 7.1.3 축 따라 이어붙이기

- 데이터를 합치는 또 다른 방법으로는 이어붙이기, 연결, 적층 등이 있는데, Numpy는 ndarray를 연결하는 concatenate 함수를 제공

In [211]:
import numpy as np
from pandas import Series

In [212]:
arr=np.arange(12).reshape((3,4))

In [213]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [214]:
np.concatenate([arr,arr],axis=1) #행을 기준, 0일경우 열을 기준

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [215]:
np.concatenate([arr,arr],axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

- Series나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 이어붙일 수 있도록 되어 있다

In [222]:
s1 = Series([0,1],index=['a','b'])
s2 = Series([2,3,4],index=['c','d','e'])
s3 = Series([5,6],index=['f','g'])

- pandas의 concat 함수는 이 사항에 대한 해답을 제공한다.
- 이 세 객체를 리스트로 묶어서 concat 함수에 전달하면 값과 색인을 연결해준다.

In [224]:
print s1
print s2
print s3

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [225]:
pd.concat([s1,s2,s3])  #axis의 값은 0으로 되어있는 경우 Series가 된다.

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [229]:
pd.concat([s1,s2,s3], axis=1) #axis의 값이 1인경우 DataFrame이 된다.
#옆으로 추가해줌

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [230]:
pd.concat([s1,s2,s3], axis=0)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [240]:
s1

a    0
b    1
dtype: int64

In [241]:
s4 = pd.concat([s1*5,s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [237]:
pd.concat([s1,s4], axis=1)

,0,1
a,0,0
b,1,5
f,NaN,5
g,NaN,6


In [239]:
pd.concat([s1,s4], axis=1, join='inner') #inner를 통해 교집합을 구함

,0,1
a,0,0
b,1,5



-------------
- 축을 직접 지정하여 merge

In [242]:
pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])

,0,1
a,0,0
c,NaN,NaN
b,1,5
e,NaN,NaN


- Series를 이어붙이기 전의 개별 Series를 구분할 수 없다는 문제가 생기는데, 이 경우 이어붙인 축에 대해 계층적 색인을 생성하여 식별이 가능하도록 할 수 있다.
- 계층적 색인을 생성하려면 keys 인자를 사용하면 된다.

In [263]:
result = pd.concat([s1,s2,s3], keys=['one','two','three'])

In [264]:
result0 = pd.concat([s1, s2, s3])

In [265]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [266]:
result0

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [267]:
result.unstack()  #row에 있는걸 column으로 올려준다.

,a,b,c,d,e,f,g
one,0,1,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2,3,4,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5,6


- Series는 axis=1로 병합할 경우 keys는 DataFrame의 칼럼 제목이 된다.

In [268]:
pd.concat([s1,s2,s3], axis=1, keys=['one','two','three'])

,one,two,three
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


- DataFrame에 대해서도 지금까지와 같은 방식으로 적용 가능

In [269]:
df1=DataFrame(np.arange(6).reshape(3,2),
              index=['a','b','c'],
              columns=['one','two'])

df2=DataFrame(5+np.arange(4).reshape(2,2),
              index=['a','c'],
              columns=['three','four'])

In [270]:
pd.concat([df1,df2], axis=1, keys=['level1','level2'])

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

- 리스트 대신 객체의 사전을 넘긴다면 샂너의 키가 keys 옵션으로 사용된다.

In [271]:
pd.concat({'level1':df1,'level2':df2}, axis=1)

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

In [272]:
pd.concat([df1,df2],axis=1,
          keys=['level1','level2'],
          names=['upper','lower'])

upper level1     level2     
lower    one two  three four
a          0   1      5    6
b          2   3    NaN  NaN
c          4   5      7    8

- DataFrame의 로우 색인이 분석에 불필요한 경우에는 어떻게 할 것인가?

In [273]:
df1=DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df2=DataFrame(np.random.randn(2,3),columns=['b','d','a'])

In [274]:
df1

,a,b,c,d
0,1.150765,-0.997174,0.046486,-0.610441
1,-0.394982,1.199915,-0.451814,-0.155385
2,-0.153514,0.011194,-0.050555,0.420211


In [275]:
df2

,b,d,a
0,1.190981,1.561488,-1.132925
1,0.097083,-1.031573,-0.044618


- ignore_index = True 옵션을 주면 된다.

In [276]:
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,1.150765,-0.997174,0.046486,-0.610441
1,-0.394982,1.199915,-0.451814,-0.155385
2,-0.153514,0.011194,-0.050555,0.420211
3,-1.132925,1.190981,NaN,1.561488
4,-0.044618,0.097083,NaN,-1.031573


In [277]:
pd.concat([df1, df2], ignore_index=True)   
#이어 붙이기 말고 불필요한 경우 다음과 같이 옵션을 주어 사용 가능

,a,b,c,d
0,1.150765,-0.997174,0.046486,-0.610441
1,-0.394982,1.199915,-0.451814,-0.155385
2,-0.153514,0.011194,-0.050555,0.420211
3,-1.132925,1.190981,NaN,1.561488
4,-0.044618,0.097083,NaN,-1.031573


In [278]:
pd.merge(df1, df2, how = 'outer')

,a,b,c,d
0,1.150765,-0.997174,0.046486,-0.610441
1,-0.394982,1.199915,-0.451814,-0.155385
2,-0.153514,0.011194,-0.050555,0.420211
3,-1.132925,1.190981,NaN,1.561488
4,-0.044618,0.097083,NaN,-1.031573


In [280]:
df1.join(df2.is)

TypeError: join() got an unexpected keyword argument 'isuffix'

## 7.1.4 겹치는 데이터 합치기

In [293]:
a = Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],
         index=['f','e','d','c','b','a'])

b = Series(np.arange(len(a),dtype=np.float64),
         index=['f','e','d','c','b','a'])

In [294]:
b[-1]=np.nan

In [296]:
len(a)

6

In [295]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [284]:
b

f     0
e     1
d     2
c     3
b     4
a   NaN
dtype: float64

In [285]:
np.where(pd.isnull(a),b,a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [291]:
b[:-2].combine_first(a[2:])
#Series의 combine_first라는 메서드는 위와 동일한 연산을 제공,
#데이터 정렬 기능까지 제공

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

- DataFrame에서도 combine_first 메서드는 사용 가능

In [287]:
df1=DataFrame({'a':[1.,np.nan,5.,np.nan],
               'b':[np.nan,2.,np.nan,6.],
               'c':range(2,18,4)})

df2=DataFrame({'a':[5.,4.,np.nan,3.,7.],
               'b':[np.nan,3.,4.,6.,8.]})

In [288]:
df1.combine_first(df2)

,a,b,c
0,1,NaN,2
1,4,2,6
2,5,4,10
3,3,6,14
4,7,8,NaN


# 7.2 재형성과 피벗

## 7.2.1 계층적 색인으로 재형성하기

- stack : 데이터의 칼럼을 로우로 피벗 또는 회전시킨다.
- unstack : 로우를 칼럼으로 피벗시킨다.

In [305]:
data=DataFrame(np.arange(6).reshape((2,3)),
               index=pd.Index(['Ohio','Colorado'],name='state'),
               columns=pd.Index(['one','two','three'],name='number'))

In [306]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [307]:
result = data.stack()

In [308]:
DataFrame(result)

0
state    number   
Ohio     one     0
         two     1
         three   2
Colorado one     3
         two     4
         three   5

In [309]:
result.unstack()
 #unstack을 사용하면 계층적 색인을 DataFrame으로 얻어올 수 있다.

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [311]:
result.unstack(0)
#인자를 레벨 이름이나 숫자를 전달해 끄집어낼 단계를  지정할 수 있다.

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [312]:
result.unstack(1)
#가장 바깥족 Ohio, colorado 같은게 row로

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [79]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


- 해당 레벨에 있는 모든 값이 하위 그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길 수 있다.

In [314]:
s1 = Series([0,1,2,3],index=['a','b','c','d'])
s2 = Series([4,5,6],index=['c','d','e'])

In [315]:
data2 = pd.concat([s1,s2],keys=['one','two'])

In [317]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,NaN
two,NaN,NaN,4,5,6


In [318]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: float64

In [319]:
DataFrame(data2.unstack().stack(dropna=False))  #na값을 다시 살린 경우

0
one a   0
    b   1
    c   2
    d   3
    e NaN
two a NaN
    b NaN
    c   4
    d   5
    e   6

- DataFrame을 unstack()할 때, unstack() 레벨은 결과에서 가장 낮은 단계가 된다.

In [322]:
df = DataFrame({'left':result,'right':result+5},
               columns=pd.Index(['left','right'],name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [86]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [87]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

## 7.2.2 피버팅으로 데이터 나열 방식 바꾸기

- CSV 파일에 여러개의 시계열 데이터를 저장하는 일반적인 방법은 시간순으로 나열하는 방법이다.

In [323]:
data = pd.read_csv('https://raw.githubusercontent.com/pydata/pydata-book/master/ch07/macrodata.csv') 
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))

ldata = data.stack().reset_index().rename(columns={0: 'value'})
wdata = ldata.pivot('date', 'item', 'value')

In [325]:
data = pd.read_csv('https://raw.githubusercontent.com/pydata/pydata-book/master/ch07/macrodata.csv') 
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))

#ldata = data.stack().reset_index().rename(columns={0: 'value'})
wdata = ldata.pivot('date', 'item', 'value')

print wdata

item        infl    realgdp  unemp
date                              
1959-03-31  0.00   2710.349    5.8
1959-06-30  2.34   2778.801    5.1
1959-09-30  2.74   2775.488    5.3
1959-12-31  0.27   2785.204    5.6
1960-03-31  2.31   2847.699    5.2
1960-06-30  0.14   2834.390    5.2
1960-09-30  2.70   2839.022    5.6
1960-12-31  1.21   2802.616    6.3
1961-03-31 -0.40   2819.264    6.8
1961-06-30  1.47   2872.005    7.0
1961-09-30  0.80   2918.419    6.8
1961-12-31  0.80   2977.830    6.2
1962-03-31  2.26   3031.241    5.6
1962-06-30  0.13   3064.709    5.5
1962-09-30  2.11   3093.047    5.6
1962-12-31  0.79   3100.563    5.5
1963-03-31  0.53   3141.087    5.8
1963-06-30  2.75   3180.447    5.7
1963-09-30  0.78   3240.332    5.5
1963-12-31  2.46   3264.967    5.6
1964-03-31  0.13   3338.246    5.5
1964-06-30  0.90   3376.587    5.2
1964-09-30  1.29   3422.469    5.0
1964-12-31  2.05   3431.957    5.0
1965-03-31  1.28   3516.251    4.9
1965-06-30  2.54   3563.960    4.7
1965-09-30  0.89   3

In [89]:
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


- MySQL 같은 관계형 데이터베이스는 데이블에 데이터가 추가되거나 삭제되면 item 칼럼에 별게의 값을 넣거나 빼는 방식으로 고정된 스키마에 데이터를 저장
- 기본키가 관계무결성을 제공하며 쉬운 조인연산과 프로그램에 의한 질의를 가능하게 해준다
- DataFrame의 pivot 메서드가 이런 변형을 지원

In [90]:
pivoted=ldata.pivot('date','item','value')

In [91]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


- pivot 메서드의 처음 두 인자는 로우나 칼럼 색인으로 사용될 칼럼 이름이고 마지막 value는 DataFrame에 채워 넣을 값을 담고 있는 칼럼

In [92]:
ldata['value2']=np.random.randn(len(ldata))

In [93]:
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,-0.332875
1,1959-03-31,infl,0.000,0.450301
2,1959-03-31,unemp,5.800,1.083705
3,1959-06-30,realgdp,2778.801,-0.347095
4,1959-06-30,infl,2.340,-0.130802
5,1959-06-30,unemp,5.100,-0.125467
6,1959-09-30,realgdp,2775.488,0.629006
7,1959-09-30,infl,2.740,-0.376932
8,1959-09-30,unemp,5.300,0.418544
9,1959-12-31,realgdp,2785.204,0.389674


- 마지막 인자를 생략해서 계층적 칼럼을 가지는 DataFrame을 얻을 수 있다.

In [94]:
pivoted=ldata.pivot('date','item')

In [95]:
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.450301 -0.332875  1.083705
1959-06-30  2.34  2778.801   5.1 -0.130802 -0.347095 -0.125467
1959-09-30  2.74  2775.488   5.3 -0.376932  0.629006  0.418544
1959-12-31  0.27  2785.204   5.6  0.570970  0.389674 -0.881622
1960-03-31  2.31  2847.699   5.2  0.363359  0.668618 -0.061841

In [96]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


- pivot 메서드는 단지 set_index를 사용해서 계층적 색인을 만들고 unstack 메서드를 이용해서 형태를 변경하는 단축키 같은 메서드

In [97]:
unstacked = ldata.set_index(['date','item']).unstack('item')

In [98]:
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.450301 -0.332875  1.083705
1959-06-30  2.34  2778.801   5.1 -0.130802 -0.347095 -0.125467
1959-09-30  2.74  2775.488   5.3 -0.376932  0.629006  0.418544
1959-12-31  0.27  2785.204   5.6  0.570970  0.389674 -0.881622
1960-03-31  2.31  2847.699   5.2  0.363359  0.668618 -0.061841
1960-06-30  0.14  2834.390   5.2 -2.103399 -0.146552 -0.884151
1960-09-30  2.70  2839.022   5.6 -0.741259 -0.128303 -0.059650

# 7.3 데이터 변형

## 7.3.1 중복 제거하기

In [327]:
data = DataFrame({'k1':['one']*3+['two']*4,'k2':[1,1,2,3,3,4,4,]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [328]:
data.duplicated() # 중복되는 값 확인
#여러개 중복이면 맨위에꺼빼고 다 drop

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [329]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [331]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [332]:
data.drop_duplicates(['k1','k2'],take_last=True)

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


## 7.3.2 함수나 매핑 이용해 데이터 변형하기

In [333]:
data = DataFrame({'food':['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','nova lox'],
                  'ounces':[4,3,12,6,7.5,8,3,5,6]})

In [334]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [335]:
meat_to_animal={
    'bacon':'pig',
    'pulled pork':'pig',
    'pastrami': 'cow',
    'corned beef':'cow',
    'honey ham':'pig',
    'nova lox':'salmon'
}

- Seires의 map 메서드는 사전류의 객체나 어떤 함수를 받을 수 있는데, 이 데이터는 육류의 이름에 대.소문자가 섞여 있는 사소한 문제가 있으므로 소문자로 모두 변경

In [336]:
data['animal']=data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [338]:
data['food'].map(lambda x:meat_to_animal[x.lower()])
#x는 column의 원소들

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

## 7.3.3 값 치환하기

- replace 메서드는 같은 작업에 대해서 좀 더 간단하고 유연한 방법을 제공한다.

In [339]:
data = Series([1.,-999.,2.,-999.,-1000.,3])
data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: float64

In [343]:
data.replace(-999, np.nan)
#-999값은 nan으로 치환

0       1
1     NaN
2       2
3     NaN
4   -1000
5       3
dtype: float64

In [112]:
data.replace([-999,-1000],[np.nan,0])

0     1
1   NaN
2     2
3   NaN
4     0
5     3
dtype: float64

In [345]:
data.replace({-999:np.nan,-1000:0}) #사전을 이용해도 된다.

0     1
1   NaN
2     2
3   NaN
4     0
5     3
dtype: float64

## 7.3.4 축 색인 이름바꾸기

In [347]:
data = DataFrame(np.arange(12).reshape((3,4)),
              index=['Ohio','Colorado','New York'],
              columns=['one','two','three','four'])

In [348]:
data.index.map(str.upper)  #인덱스는 대문자로

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [349]:
data.index=data.index.map(str.upper) #이런식으로 대입이 가능
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


- 대문자로 변경된 축 이름을 DataFrame의 index에 바로 대입할 수 있다.

In [350]:
data.rename(index=str.title,columns=str.upper)  
#column의 이름을 바꿀 수 있다.

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [352]:
data.rename(index={'OHIO':'INDIANA'},
            columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


- 원래 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메서드를 사용

In [353]:
_=data.rename(index={'OHIO':'INDIANA'},inplace=True)
 #원본 데이터로 바로 변경하려면, 
    #inplace = True를 옵션으로 넘겨준다.
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


## 7.3.5 개별화와 양자화

In [354]:
ages = [20,22,25,27,21,23,37,31,61,45,41,32]

- pandas의 cut 함수를 이용해서 그룹으로 나눌 수 있다.

In [355]:
bins = [18,25,35,60,100]

In [356]:
cats = pd.cut(ages,bins)

In [357]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

- pandas에서 반환하는 객체는 Categorical 객체라는 특수한 객체인데, 이 객체는 그룹 이름이 담긴 배열이라고 생각하면 된다.
- labels 속성에 있는 ages데이터에 대한 카테고리 이름을 levels라는 배열에 내부적으로 담고 있다.

In [125]:
cats.labels

C:\Users\S525\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\core\categorical.py:420: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  warnings.warn("'labels' is deprecated. Use 'codes' instead", FutureWarning)


array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [126]:
cats.levels

C:\Users\S525\AppData\Local\Enthought\Canopy\User\lib\site-packages\pandas\core\categorical.py:472: FutureWarning: Accessing 'levels' is deprecated, use 'categories'
  warn("Accessing 'levels' is deprecated, use 'categories'", FutureWarning)


Index([u'(18, 25]', u'(25, 35]', u'(35, 60]', u'(60, 100]'], dtype='object')

In [358]:
pd.value_counts(cats)  #각 bins 별로 몇개가 들어가는지

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

- 간격을 나타내는 표기법은 괄호로 시작해서 대괄호로 끝나는데 괄호 쪽의 값은 포함하지 않고 대괄호 쪽의 값은 포함하는 간격을 나타낸다.
- right=False를 넘겨서 괄호와 대괄호의 위치를 바꿀수 있다.

In [128]:
pd.cut(ages,[18,26,36,61,100],right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [129]:
pd.cut(ages,[18,26,36,61,100],right=True)

[(18, 26], (18, 26], (18, 26], (26, 36], (18, 26], ..., (26, 36], (36, 61], (36, 61], (36, 61], (26, 36]]
Length: 12
Categories (4, object): [(18, 26] < (26, 36] < (36, 61] < (61, 100]]

In [359]:
group_names=['Youth','YoungAdult','MiddleAged','Senior']
#labelse 옵션으로 그룹의 이름을 직접 넘겨 줄 수 있다.

In [131]:
pd.cut(ages,bins,labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]


- cut 함수에 명시적으로 그룹의 경계값을 넘기지 않고 그룹의 개수를 넘겨주면 데이터에서 최소 값과 최대 값을 기준으로 균등한 길이의 그룹의 자동으로 계산한다.

---------------

In [362]:
data = np.random.rand(20)

In [363]:
data

array([ 0.07557764,  0.21863608,  0.90953758,  0.38441329,  0.08127486,
        0.95663486,  0.04387138,  0.14476865,  0.11766477,  0.13018305,
        0.62275769,  0.08497892,  0.43897158,  0.39948258,  0.64548107,
        0.75527534,  0.03114722,  0.74752639,  0.72403557,  0.2732724 ])

In [364]:
data.max()

0.95663486009392762

In [365]:
data.min()

0.031147217896063562

In [133]:
pd.cut(data,4,precision=2)

[(0.096, 0.32], (0.32, 0.54], (0.32, 0.54], (0.54, 0.76], (0.76, 0.98], ..., (0.32, 0.54], (0.76, 0.98], (0.76, 0.98], (0.32, 0.54], (0.32, 0.54]]
Length: 20
Categories (4, object): [(0.096, 0.32] < (0.32, 0.54] < (0.54, 0.76] < (0.76, 0.98]]

------------
- 이를 위한 가장 적합한 함수로 qcut이 있는데, 이는 표본 변위치를 기반으로 데이터를 나눠준다.
- cut 함수를 사용하게되념ㄴ 데이터 분산에 따라 각각의 그룹마다 데이터의 개수가 다르게 나누어지는 경우가 많다.
- qcut은 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [366]:
data = np.random.randn(1000)

In [369]:
np.random.randn?

In [370]:
cats = pd.qcut(data,4)
cats

[(0.664, 3.526], (0.664, 3.526], (0.664, 3.526], (-0.0229, 0.664], (-0.692, -0.0229], ..., (-0.0229, 0.664], (-0.692, -0.0229], (0.664, 3.526], (0.664, 3.526], (-0.692, -0.0229]]
Length: 1000
Categories (4, object): [[-3.646, -0.692] < (-0.692, -0.0229] < (-0.0229, 0.664] < (0.664, 3.526]]

In [371]:
pd.value_counts(cats)

(0.664, 3.526]       250
(-0.0229, 0.664]     250
(-0.692, -0.0229]    250
[-3.646, -0.692]     250
dtype: int64

In [378]:
b = pd.qcut(data,[0,0.1,0.5,0.9,1.])
#비율 정해줌  (변위치 값은 0~1)

In [380]:
pd.value_counts(b)

(-0.0229, 1.244]     400
(-1.267, -0.0229]    400
(1.244, 3.526]       100
[-3.646, -1.267]     100
dtype: int64

## 7.3.6 특이값 찾아내고 제외하기

In [389]:
np.random.seed(12345)
data=DataFrame(np.random.randn(1000,4))

In [390]:
data

,0,1,2,3
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.274992,0.228913,1.352917,0.886429
4,-2.001637,-0.371843,1.669025,-0.438570
5,-0.539741,0.476985,3.248944,-1.021228
6,-0.577087,0.124121,0.302614,0.523772
7,0.000940,1.343810,-0.713544,-0.831154
8,-2.370232,-1.860761,-0.860757,0.560145
9,-1.265934,0.119827,-1.063512,0.332883


In [391]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


- DataFrame의 한 칼럼에서 절대 값이 3을 초과하는 값을 찾아내자

In [392]:
col = data[3]
col[np.abs(col) > 3]   #절대값이 3보다 큰애들만 뽑아낸다.

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [393]:
col

0     -0.555730
1      0.281746
2     -1.296221
3      0.886429
4     -0.438570
5     -1.021228
6      0.523772
7     -0.831154
8      0.560145
9      0.332883
10    -0.970736
11    -0.753887
12     0.246674
13    -0.919262
14    -0.660524
15     0.670216
16    -2.304234
17     1.074623
18    -0.503087
19     0.222896
20     0.433610
21     0.850591
22     2.169461
23     0.795253
24     0.152677
25    -0.929006
26     0.980928
27     0.457002
28    -0.402827
29    -1.648985
         ...   
970   -1.460094
971    0.132262
972   -1.091849
973   -1.243078
974   -0.521155
975   -0.307670
976    1.381155
977    0.075798
978    0.102469
979    1.120895
980   -0.285176
981    0.053410
982    0.233619
983    0.604880
984   -0.335099
985    1.367213
986   -0.512542
987   -1.050942
988   -0.387002
989    1.642339
990    1.336648
991    0.153988
992   -1.466596
993    1.101972
994    0.604072
995    1.653126
996   -1.502870
997    0.133304
998   -0.303340
999    0.882755
Name: 3, dtype: float64

In [383]:
data[(np.abs(data)>3).any(1)]
#절대 값 3을 초과하는 값이 들어있는 모든 로우를 선택하려면 any 메서드를 활용

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [398]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [384]:
data[np.abs(data)>3]=np.sign(data)*3
data.describe()   #한번에 여러 개의 통계결과를 만들어 낸다.

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000
